## Wikipedia text 정제 및 문장분리
### 20231110

In [1]:
import pandas as pd
import numpy as np
import os
import re
import stanza
# Stanza 초기화
stanza.download('id')  # 영어 모델 다운로드
nlp = stanza.Pipeline('id', processors='tokenize')
from tqdm import tqdm

2023-11-11 14:56:55 INFO: Downloading default packages for language: id (Indonesian) ...
2023-11-11 14:56:55 INFO: File exists: C:\Users\USER\stanza_resources\id\default.zip
2023-11-11 14:56:58 INFO: Finished downloading models and saved to C:\Users\USER\stanza_resources.
2023-11-11 14:56:58 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-11 14:56:58 WARNING: Language id package default expects mwt, which has been added
2023-11-11 14:56:58 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-11-11 14:56:58 INFO: Using device: cpu
2023-11-11 14:56:58 INFO: Loading: tokenize
2023-11-11 14:56:58 INFO: Loading: mwt
2023-11-11 14:56:58 INFO: Done loading processors!


In [2]:
os.getcwd()

'c:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\위키 데이터'

In [4]:
df = pd.read_csv('C:\\Users\\USER\\Desktop\\wiki수동합치기2.csv')
df

,title,date,text
0,Konfusianisme,2023-07-17T03:00:00Z,{{tak netral}} {{rapikan}} {{Infobox Chinese |...
1,Asia,2023-07-19T12:11:07Z,{{kegunaanlain}} {{Infobox continent |title = ...
2,Kanji,2022-12-21T05:17:27Z,{{disambiginfo}} {{Infobox Writing system |nam...
3,Aksara Han,2022-03-04T08:32:23Z,{{Infobox writing system |name=Aksara Han <br>...
4,Biksu,2023-05-29T06:39:19Z,{{more citations needed|date=Januari 2023}} [[...
...,...,...,...
18160,oasis (seri televisi korea selatan),2023-07-09T03:29:01Z,genre writer jung hyung soo director han hee s...
18161,templat:infobox korean,2023-07-11T05:50:00Z,yes yes headercolor float collapse pic piccap ...
18162,daftar katedral di korea utara,2023-07-18T11:17:41Z,jmpl ka katedral changchung di pyongyang korea...
18163,banjir korea selatan 2023,2023-07-17T18:10:10Z,damages missing sembilan fatalities empat satu...


In [5]:
df = df.drop_duplicates(subset = 'title')
df

,title,date,text
0,Konfusianisme,2023-07-17T03:00:00Z,{{tak netral}} {{rapikan}} {{Infobox Chinese |...
1,Asia,2023-07-19T12:11:07Z,{{kegunaanlain}} {{Infobox continent |title = ...
2,Kanji,2022-12-21T05:17:27Z,{{disambiginfo}} {{Infobox Writing system |nam...
3,Aksara Han,2022-03-04T08:32:23Z,{{Infobox writing system |name=Aksara Han <br>...
4,Biksu,2023-05-29T06:39:19Z,{{more citations needed|date=Januari 2023}} [[...
...,...,...,...
18160,oasis (seri televisi korea selatan),2023-07-09T03:29:01Z,genre writer jung hyung soo director han hee s...
18161,templat:infobox korean,2023-07-11T05:50:00Z,yes yes headercolor float collapse pic piccap ...
18162,daftar katedral di korea utara,2023-07-18T11:17:41Z,jmpl ka katedral changchung di pyongyang korea...
18163,banjir korea selatan 2023,2023-07-17T18:10:10Z,damages missing sembilan fatalities empat satu...


In [6]:
def process_test(text):

    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<ref[^<]*</ref>", "", text)
    text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)
    text = re.sub(r"{{[^}]*}}", "", text)
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"<[^>]*>", "", text)
    #text = re.sub(r"\|\d+px", "", text) #remove image px
    text = re.sub(r"==\s*([^=]+)\s*=+", "", text) #소제목 제거
    text = re.sub(r"\=", "", text)
    text = re.sub(r"\[http[^\]]+\]", "", text) #http 제거
    text = re.sub(r"\[\[Kategori:[^\]]+\]\]","", text) #카테고리 제거
    text = re.sub(r"\{", "", text)
    text = re.sub(r"\}", "", text)
    text = re.sub(r"\\'","", text) #wiki에서 글씨체 효과 제거
    text = re.sub(r"\'''", "", text) #wiki에서 bold처리 표현식 제거
    text = re.sub(r"\''", "", text) #wiki에서 이탤릭체 표현식 제거
    text = re.sub(r"\#", "", text) #wiki 숫자 표시 제거
    text = re.sub(r"\*", "", text) #wiki 점 제거
    text = re.sub(r"\[", "", text)
    text = re.sub(r"\]", "", text)
    #text = re.sub(r"\|[^|]+\|", "|", text)
    #text = re.sub(r"\|.*? |\|\d+px", "", text) #remove | and image px
    text = re.sub(r'jmpl\|', '', text)
    text = re.sub(r'ka\|', '', text)
    text = re.sub(r'\d+px\|', '', text)
    text = re.sub(r'kiri\|', '', text)
    text = re.sub(r'thumb\|', '', text)
    text = re.sub(r'lurus\|', '', text)
    text = re.sub(r'left\|', '', text)
    text = re.sub(r'right\|', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',"",text)
    text = re.sub(r"\|.*?   |\|\d+px","",text)
    text = re.sub(r"\|.*?  |\|\d+px","",text)
    text = re.sub(r"\|.*? |\|\d+px","",text)
    text = re.sub(r"\|.*?|\|\d+px","",text)
    text = re.sub(r"\d+px", "",text) #remove image px
    text = re.sub(r"[가-힣]+","",text) #remove 한글
    text = re.sub(r"[一-龥]+","",text) #remove 한자
    text = re.sub(r"\([^)]*\)","",text) #remove 소괄호, 소괄호 속 내용
    text = re.sub(r"http\S+", "", text) #remove http
    text = re.sub(r"\!","",text)
    text = re.sub(r'.*?\.jpg\s*', '', text) #jpg 앞쪽 문장 모두 제거
    text = re.sub(r'jpg', '', text)
    text = re.sub(r'Berkas:.*',"",text) #Berkas: 로 문장 시작 시 모두 제거
    text = re.sub(r'\s*section at the bottom of the .*$','',text)
    text = re.sub(r'text = re.sub(r"[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF『』、。《》]", "", text)','',text) #remove 일본어
    text = re.sub(r'-->',"",text)
    text = re.sub(r'-',"",text)
    text = re.sub(r'::',"",text)
    text = re.sub(r'《',"",text)
    text = re.sub(r'》',"",text)
    text = re.sub(r'『',"",text)
    text = re.sub(r'』',"",text)
    text = re.sub(r'&x7C;',"",text)
    

    return text

In [7]:
# NaN 값을 처리합니다. 예를 들어, NaN 값을 빈 문자열로 대체합니다.
df['text'] = df['text'].fillna('')

# 모든 값을 문자열로 변환합니다.
df['text'] = df['text'].astype(str)


C:\Users\USER\AppData\Local\Temp\ipykernel_3492\2379213588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].fillna('')
C:\Users\USER\AppData\Local\Temp\ipykernel_3492\2379213588.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].astype(str)


In [8]:
df['title'] = df['title'].fillna('')

# 모든 값을 문자열로 변환합니다.
df['title'] = df['title'].astype(str)

C:\Users\USER\AppData\Local\Temp\ipykernel_3492\460863158.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].fillna('')
C:\Users\USER\AppData\Local\Temp\ipykernel_3492\460863158.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].astype(str)


In [9]:
df['cleansing_test'] = df['text'].apply(process_test)
df

KeyboardInterrupt: 

##### 정제된 문장 분리 및 토크나이징

In [ ]:
def tokenizing(df):
    max_length=5
    new_rows = []
    index=0
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        index+=1
        article_text = row['cleansing_test']
        doc = nlp(article_text)
        sen_id = 1  # 문장 ID를 1로 초기화
        for sentence in doc.sentences:
            new_row = row.copy()
            new_row['Sentence'] = sentence.text
            new_row['Token']= [token.text for token in sentence.tokens]
            new_row['Word_Count'] = len(new_row['Token'])
            new_row['Doc_ID']='20230816_wikidata_'+new_row['title']+'_000001'
            new_row['Sen_ID'] = new_row['Doc_ID']+'_sen'+str('{:06d}'.format(sen_id))
            new_row['Pub_Date']=new_row['date'][:10]
            new_row['Title']=new_row['title']
            new_row['Pub_Subj']=new_row['title']
            new_rows.append(new_row)
            sen_id += 1  # 문장 ID 증가

    new_df = pd.DataFrame(new_rows)
    new_df['Tokenized_Sentence']= new_df['Token'].apply(' '.join)
    new_df['Col_Date']='2023-08-10'
    new_df['Pub_Type']='Wikipedia'
    new_df['Text']=new_df['cleansing_test']
    new_df['Filename']='C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NIA AI 데이터셋 구축 - General/06-수집데이터/위키 데이터/rawdata/idwiki-20230720-pages-articles-multistream.xml'

    new_df.drop_duplicates(subset='Sentence', inplace=True)
    new_df.reset_index(drop=True, inplace=True)
    new_df = new_df[['Doc_ID', 'Filename', 'Title', 'Pub_Type', 'Pub_Subj', 'Pub_Date', 'Col_Date', 'Sen_ID', 'Word_Count', 'Text', 'Sentence','Tokenized_Sentence' , 'Token']]
    new_df = new_df[new_df['Token'].apply(lambda x: len(x) > max_length)]
    return new_df

In [ ]:
df_test= tokenizing(df)
df_test

Processing rows: 100%|██████████| 12356/12356 [12:03<00:00, 17.07it/s]


,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000001,29,upright1.15 Konfusianisme umumnya memang ti...,upright1.15 Konfusianisme umumnya memang tidak...,upright1.15 Konfusianisme umumnya memang tidak...,"[upright1.15, Konfusianisme, umumnya, memang, ..."
1,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000002,15,upright1.15 Konfusianisme umumnya memang ti...,Namun di Indonesia Konghucu diakui sebagai sal...,Namun di Indonesia Konghucu diakui sebagai sal...,"[Namun, di, Indonesia, Konghucu, diakui, sebag..."
2,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000003,13,upright1.15 Konfusianisme umumnya memang ti...,Konghucu sebagai agama digagas Kang Youwei men...,Konghucu sebagai agama digagas Kang Youwei men...,"[Konghucu, sebagai, agama, digagas, Kang, Youw..."
3,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000004,35,upright1.15 Konfusianisme umumnya memang ti...,Namun gagasan Kang Youwei tampaknya tidak dite...,Namun gagasan Kang Youwei tampaknya tidak dite...,"[Namun, gagasan, Kang, Youwei, tampaknya, tida..."
4,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000005,37,upright1.15 Konfusianisme umumnya memang ti...,"Dalam bahasa Tionghoa, ajaran Konghucu dikenal...","Dalam bahasa Tionghoa , ajaran Konghucu dikena...","[Dalam, bahasa, Tionghoa, ,, ajaran, Konghucu,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75929,20230816_wikidata_oasis (seri televisi korea s...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,oasis (seri televisi korea selatan),Wikipedia,oasis (seri televisi korea selatan),2023-07-09,2023-08-10,20230816_wikidata_oasis (seri televisi korea s...,116,genre writer jung hyung soo director han hee s...,genre writer jung hyung soo director han hee s...,genre writer jung hyung soo director han hee s...,"[genre, writer, jung, hyung, soo, director, ha..."
75930,20230816_wikidata_templat:infobox korean_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,templat:infobox korean,Wikipedia,templat:infobox korean,2023-07-11,2023-08-10,20230816_wikidata_templat:infobox korean_00000...,315,yes yes headercolor float collapse pic piccap ...,yes yes headercolor float collapse pic piccap ...,yes yes headercolor float collapse pic piccap ...,"[yes, yes, headercolor, float, collapse, pic, ..."
75931,20230816_wikidata_daftar katedral di korea uta...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,daftar katedral di korea utara,Wikipedia,daftar katedral di korea utara,2023-07-18,2023-08-10,20230816_wikidata_daftar katedral di korea uta...,53,jmpl ka katedral changchung di pyongyang korea...,jmpl ka katedral changchung di pyongyang korea...,jmpl ka katedral changchung di pyongyang korea...,"[jmpl, ka, katedral, changchung, di, pyongyang..."
75932,20230816_wikidata_banjir korea selatan 2023_00...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,banjir korea selatan 2023,Wikipedia,banjir korea selatan 2023,2023-07-17,2023-08-10,20230816_wikidata_banjir korea selatan 2023_00...,320,damages missing sembilan fatalities empat satu...,damages missing sembilan fatalities empat satu...,damages missing sembilan fatalities empat satu...,"[damages, missing, sembilan, fatalities, empat..."


In [ ]:
df_test['Word_Count'].sum()

2235631

In [ ]:
os.getcwd()

'c:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터'

In [ ]:
dfd = df_test.to_csv('wikiall20231110문장분리전.csv', index = False, encoding = 'utf-8-sig')

### making new column

In [3]:
import pandas as pd
import numpy as np
import os
import re
import stanza
# Stanza 초기화
stanza.download('id')  # 영어 모델 다운로드
nlp = stanza.Pipeline('id', processors='tokenize')
from tqdm import tqdm

2023-11-11 16:45:39 INFO: Downloading default packages for language: id (Indonesian) ...
2023-11-11 16:45:39 INFO: File exists: C:\Users\USER\stanza_resources\id\default.zip
2023-11-11 16:45:42 INFO: Finished downloading models and saved to C:\Users\USER\stanza_resources.
2023-11-11 16:45:42 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-11 16:45:42 WARNING: Language id package default expects mwt, which has been added
2023-11-11 16:45:42 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-11-11 16:45:42 INFO: Using device: cpu
2023-11-11 16:45:42 INFO: Loading: tokenize
2023-11-11 16:45:42 INFO: Loading: mwt
2023-11-11 16:45:42 INFO: Done loading processors!


In [4]:
path = 'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\원천데이터\\위키데이터\\20231110기준wikiall'
os.chdir(path)

In [51]:
df = pd.read_csv('wikialldata_1.csv')
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000001,29,upright1.15 Konfusianisme umumnya memang ti...,upright1.15 Konfusianisme umumnya memang tidak...,upright1.15 Konfusianisme umumnya memang tidak...,"['upright1.15', 'Konfusianisme', 'umumnya', 'm..."
1,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000002,15,upright1.15 Konfusianisme umumnya memang ti...,Namun di Indonesia Konghucu diakui sebagai sal...,Namun di Indonesia Konghucu diakui sebagai sal...,"['Namun', 'di', 'Indonesia', 'Konghucu', 'diak..."
2,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000003,13,upright1.15 Konfusianisme umumnya memang ti...,Konghucu sebagai agama digagas Kang Youwei men...,Konghucu sebagai agama digagas Kang Youwei men...,"['Konghucu', 'sebagai', 'agama', 'digagas', 'K..."
3,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000004,35,upright1.15 Konfusianisme umumnya memang ti...,Namun gagasan Kang Youwei tampaknya tidak dite...,Namun gagasan Kang Youwei tampaknya tidak dite...,"['Namun', 'gagasan', 'Kang', 'Youwei', 'tampak..."
4,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000005,37,upright1.15 Konfusianisme umumnya memang ti...,"Dalam bahasa Tionghoa, ajaran Konghucu dikenal...","Dalam bahasa Tionghoa , ajaran Konghucu dikena...","['Dalam', 'bahasa', 'Tionghoa', ',', 'ajaran',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,20230816_wikidata_The Book of Us: Gravity_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,The Book of Us: Gravity,Wikipedia,The Book of Us: Gravity,2023-01-04,2023-08-10,20230816_wikidata_The Book of Us: Gravity_0000...,29,The Book of Us: Gravity adalah album mini k...,The Book of Us: Gravity adalah album mini keli...,The Book of Us : Gravity adalah album mini kel...,"['The', 'Book', 'of', 'Us', ':', 'Gravity', 'a..."
49996,20230816_wikidata_The Book of Us: Gravity_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,The Book of Us: Gravity,Wikipedia,The Book of Us: Gravity,2023-01-04,2023-08-10,20230816_wikidata_The Book of Us: Gravity_0000...,6,The Book of Us: Gravity adalah album mini k...,Diadaptasi dari situs resmi grup.,Diadaptasi dari situs resmi grup .,"['Diadaptasi', 'dari', 'situs', 'resmi', 'grup..."
49997,20230816_wikidata_The Book of Us: Entropy_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,The Book of Us: Entropy,Wikipedia,The Book of Us: Entropy,2021-04-09,2023-08-10,20230816_wikidata_The Book of Us: Entropy_0000...,29,The Book of Us: Entropy adalah album studio...,The Book of Us: Entropy adalah album studio ke...,The Book of Us : Entropy adalah album studio k...,"['The', 'Book', 'of', 'Us', ':', 'Entropy', 'a..."
49998,20230816_wikidata_The Book of Us: Entropy_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,The Book of Us: Entropy,Wikipedia,The Book of Us: Entropy,2021-04-09,2023-08-10,20230816_wikidata_The Book of Us: Entropy_0000...,12,The Book of Us: Entropy adalah album studio...,"Singel utama ""Sweet Chaos"" dirilis pada hari y...","Singel utama "" Sweet Chaos "" dirilis pada hari...","['Singel', 'utama', '""', 'Sweet', 'Chaos', '""'..."


In [52]:
import ast  # 문자열을 리스트로 변환하는 데 사용

# 빈 데이터프레임 리스트 초기화
temp_dfs = []

# Token 열의 문자열을 실제 리스트로 변환
df['Token'] = df['Token'].apply(ast.literal_eval)

# 가장 많은 토큰을 가진 행의 토큰 개수를 찾음
max_tokens = df['Token'].apply(len).max()

# 필요한 만큼의 새 열을 미리 생성
for i in range(max_tokens):
    df[f'Column_{i}'] = None

# 각 행에 대해 반복
for idx in tqdm(df.index):
    # 해당 행의 토큰 리스트
    tokens = df.loc[idx, 'Token']

    # 각 토큰에 대해 반복
    for i, token in enumerate(tokens):
        # 해당 토큰을 적절한 열에 할당
        df.at[idx, f'Column_{i}'] = token

C:\Users\USER\AppData\Local\Temp\ipykernel_12708\3885761950.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Column_{i}'] = None
C:\Users\USER\AppData\Local\Temp\ipykernel_12708\3885761950.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Column_{i}'] = None
C:\Users\USER\AppData\Local\Temp\ipykernel_12708\3885761950.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

In [53]:
df.reset_index(drop=True, inplace=True)
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,...,Column_1807,Column_1808,Column_1809,Column_1810,Column_1811,Column_1812,Column_1813,Column_1814,Column_1815,Column_1816
0,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000001,29,upright1.15 Konfusianisme umumnya memang ti...,...,None,None,None,None,None,None,None,None,None,None
1,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000002,15,upright1.15 Konfusianisme umumnya memang ti...,...,None,None,None,None,None,None,None,None,None,None
2,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000003,13,upright1.15 Konfusianisme umumnya memang ti...,...,None,None,None,None,None,None,None,None,None,None
3,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000004,35,upright1.15 Konfusianisme umumnya memang ti...,...,None,None,None,None,None,None,None,None,None,None
4,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000005,37,upright1.15 Konfusianisme umumnya memang ti...,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,20230816_wikidata_The Book of Us: Gravity_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,The Book of Us: Gravity,Wikipedia,The Book of Us: Gravity,2023-01-04,2023-08-10,20230816_wikidata_The Book of Us: Gravity_0000...,29,The Book of Us: Gravity adalah album mini k...,...,None,None,None,None,None,None,None,None,None,None
49996,20230816_wikidata_The Book of Us: Gravity_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,The Book of Us: Gravity,Wikipedia,The Book of Us: Gravity,2023-01-04,2023-08-10,20230816_wikidata_The Book of Us: Gravity_0000...,6,The Book of Us: Gravity adalah album mini k...,...,None,None,None,None,None,None,None,None,None,None
49997,20230816_wikidata_The Book of Us: Entropy_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,The Book of Us: Entropy,Wikipedia,The Book of Us: Entropy,2021-04-09,2023-08-10,20230816_wikidata_The Book of Us: Entropy_0000...,29,The Book of Us: Entropy adalah album studio...,...,None,None,None,None,None,None,None,None,None,None
49998,20230816_wikidata_The Book of Us: Entropy_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,The Book of Us: Entropy,Wikipedia,The Book of Us: Entropy,2021-04-09,2023-08-10,20230816_wikidata_The Book of Us: Entropy_0000...,12,The Book of Us: Entropy adalah album studio...,...,None,None,None,None,None,None,None,None,None,None


In [55]:
dc = df.to_csv('wikidata_1_column포함.csv', index=False, encoding='utf-8-sig')